## Read Data

In [1]:
import pandas as pd
fp = "../../data/online_retail_Q1_2010.parquet"
df = pd.read_parquet(fp)

In [2]:
df.sample(n=5)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
43635,497795,84993B,75 BLACK PETIT FOUR CASES,24,2010-02-12 12:51:00,0.42,16954.0,United Kingdom
99045,503148,21246,RED SPOTTY BIG BOWL,6,2010-03-30 12:17:00,4.95,15715.0,United Kingdom
25823,C495767,ADJUST,Adjustment by john on 26/01/2010 16,-1,2010-01-26 16:39:00,50.76,12696.0,Finland
79647,501306,22281,EASTER TREE YELLOW BIRDS,2,2010-03-15 17:29:00,12.72,NaN,United Kingdom
56835,499073,21380,WOODEN HAPPY BIRTHDAY GARLAND,1,2010-02-24 15:23:00,5.91,NaN,United Kingdom


In [3]:
def valid_date_time(x):
    try:
        pd.to_datetime(x)
        return True
    except Exception as e:
        return False
valid_datetime = df["InvoiceDate"].apply(valid_date_time)

In [4]:
df = df[valid_datetime]

In [5]:
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

## Identify the columns with mixed types

In [6]:
cols = df.columns.tolist()
[cols[i] for i in [3,5,6]]

['Quantity', 'Price', 'Customer ID']

## Noise Filter #1 Definition

In [7]:
valid_cust = ~df["Customer ID"].isna()
valid_desc = ~df["Description"].isna()
valid_stock_code = ~df["Description"].isna()

## Apply Noise Filter # 1

In [8]:
noise_filter_1 = valid_cust & valid_desc
df = df[noise_filter_1].reset_index(drop=True)

## Noise Filter #2 Definition

In [9]:
def good_quantity_record_check(x):
    try:
        f_x = float(x)
        if f_x > 0:
            return True
        else:
            return False # returned purchase
    except :
        return False


## Apply Noise Filter #2

In [10]:
good_quantity_records = df["Quantity"].apply(good_quantity_record_check)
df = df[good_quantity_records]
df = df.reset_index(drop=True)

## Noise Filter #3 Definition

In [11]:
return_or_bank_charges = (df.Price == "BANK CHARGES") | (df.Price == "ADJUST")
valid_purchases = ~ return_or_bank_charges

## Apply Noise Filter #3

In [12]:
df = df[valid_purchases]
df = df.reset_index(drop=True)

## Noise Filter #4 Definition

In [13]:
def not_test_product(x):
    if "TEST" in x:
        return False
    elif x in ["ADJUST", "BANK CHARGES", "C2", "M"]:
        return False
    else:
        return True

## Apply Noise Filter #4

In [14]:
valid_products = df["StockCode"].apply(not_test_product)
df = df[valid_products]
df = df.reset_index(drop=True)

In [15]:
cols

['Invoice',
 'StockCode',
 'Description',
 'Quantity',
 'InvoiceDate',
 'Price',
 'Customer ID',
 'Country']

## Define Noise Filter #5

In [16]:

attr_types = {"Invoice": str, "StockCode": str, "Description": str,\
             "Quantity": float, "InvoiceDate": 'datetime64[ns]', "Price": float,\
             "Customer ID": str, "Country": str}
df = df.astype(attr_types)
Q1_2010 = (df["InvoiceDate"].dt.year == 2010) & (df["InvoiceDate"].dt.quarter == 1)

## Apply Noise Filter #5

In [17]:
df = df[Q1_2010]
df = df.reset_index(drop=True)

## Transformation Definition

In [18]:
df["item_total"] = df["Quantity"] * df["Price"]
dsbysc = df.groupby([df.InvoiceDate.dt.day_of_year, df.StockCode])
dsbysc = dsbysc["item_total"].sum().to_frame().reset_index()
dfQ1_PA = dsbysc.pivot(index="InvoiceDate", columns="StockCode", values="item_total").fillna(0)

## Result

In [19]:
dfQ1_PA

StockCode,10002,10120,10123C,10124A,10125,10133,10134,10135,10138,11001,...,90214L,90214M,90214N,90214O,90214P,90214R,90214S,90214V,PADS,POST
InvoiceDate,,,,,,,,,,,,,,,,,,,,,
4,2.55,6.3,0.0,0.0,0.0,0.00,0.00,1.25,0.00,0.00,...,0.0,0.0,2.5,0.0,0.00,0.0,1.25,0.0,0.0,0.0
5,0.00,0.0,0.0,0.0,0.0,0.00,0.00,12.50,0.00,3.38,...,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0
6,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0
7,0.00,0.0,0.0,0.0,0.0,108.00,212.00,0.00,0.00,27.04,...,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,19.0
8,10.20,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,477.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,11.05,0.0,0.0,0.0,0.0,0.00,0.00,25.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,15.0
87,0.00,0.0,0.0,0.0,5.1,0.85,1.25,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0
88,0.00,0.0,0.0,0.0,17.0,0.00,0.00,0.00,5.04,0.00,...,0.0,0.0,0.0,0.0,1.25,0.0,0.00,0.0,0.0,0.0


## Save Prepared Data to Disk

In [20]:
fp = "../../data/retail_q1_sales_2010_summary.parquet"
dfQ1_PA.to_parquet(fp, index=False)

In [21]:
from ontology.kmds_ontology import *
from tagging.tag_types import ExploratoryTags

In [22]:
kaw = KnowledgeApplicationWorkflow("retail_customer_modelling")

In [23]:
observation_count = 1
e1 = ExploratoryObservation()

In [24]:
e1.finding = "Quantity, Price and Customer ID attributes have mixed types"
e1.finding_seq = observation_count
e1.exploratory_observation_type = ExploratoryTags.DATA_QUALITY_OBSERVATION.value

In [25]:
observation_count += 1

In [26]:
e2 = ExploratoryObservation()
e2.finding = "Customer ID, Description and Stock Code have null values"
e2.finding_seq = observation_count
e2.exploratory_observation_type = ExploratoryTags.DATA_QUALITY_OBSERVATION.value

In [27]:
observation_count += 1
e3 = ExploratoryObservation()
e3.finding = "Some Quantities are not numbers"
e3.finding_seq = observation_count
e3.exploratory_observation_type = ExploratoryTags.DATA_QUALITY_OBSERVATION.value

In [28]:
observation_count += 1
e4 = ExploratoryObservation()
e4.finding = "Some Prices are not numbers, these correspond to transactions that are some kind of adjustment - like returns, or,\
denoting some kind of payment adjustment. "
e4.finding_seq = observation_count
e4.exploratory_observation_type = ExploratoryTags.DATA_QUALITY_OBSERVATION.value

In [29]:
observation_count += 1
e5 = ExploratoryObservation()
e5.finding = "Some products are not valid products. For example, some of these products seem to be created by software testers "
e5.finding_seq = observation_count
e5.exploratory_observation_type = ExploratoryTags.DATA_QUALITY_OBSERVATION.value

In [30]:
observation_count += 1
e6 = ExploratoryObservation()
e6.finding = "For this task, the period considered is first quarter of 2010, other data are not relevant for this report "
e6.finding_seq = observation_count
e6.exploratory_observation_type = ExploratoryTags.RELEVANCE_OBSERVATION.value